# Fine-Tuning BERT on MultiNLI Dataset for NLI task with two labels (Entailment and Non-entailment)

Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True, timeout_ms = 0)

Mounted at /content/drive


In [2]:
base_dir = '/content/drive/My Drive/'

To be able to share the model with the community and use Trainer API, one should have an authentication token from the Hugging Face

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Imports

Installing packages

In [4]:
! pip install datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


Importing libraries

In [5]:
import torch
import numpy as np
import os
import datetime
from copy import deepcopy

from transformers import (RobertaTokenizer,
                          RobertaForSequenceClassification,
                          Trainer,
                          TrainingArguments,
                          TrainerCallback)
from datasets import (load_dataset,
                      load_metric,
                      load_dataset_builder,
                      ClassLabel,
                      Value,
                      Features)

In [6]:
BATCH_SIZE = 32

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [ ]:
ds_builder = load_dataset_builder("glue", "mnli")
ds_builder.info.features

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'idx': Value(dtype='int32', id=None)}

Using only train and validation data

In [8]:
train_data, val_m_data, val_mm_data = load_dataset("glue", "mnli", split=['train', 'validation_matched', 'validation_mismatched'])

print("Train dataset:\n", train_data)
print("Validation matched dataset:\n", val_m_data)
print("Validation mismatched dataset:\n", val_mm_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Train dataset:
 Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 392702
})
Validation matched dataset:
 Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 9815
})
Validation mismatched dataset:
 Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 9832
})


In [9]:
train_data.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [10]:
# Example
print(train_data[0])

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'hypothesis': 'Product and geography are what make cream skimming work. ', 'label': 1, 'idx': 0}


In [11]:
metric = load_metric('glue', "mnli")

<ipython-input-11-17bf5bf84e68>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

## Preprocessing the data

Replacing neutral (1) and contradiction (2) labels with 1 (non-entailment).

In [12]:
new_features = train_data.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

train_data = train_data.cast(new_features)
val_m_data = val_m_data.cast(new_features)
val_mm_data = val_mm_data.cast(new_features)

print(train_data.features)

Casting the dataset:   0%|          | 0/392702 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9815 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/9832 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None), 'idx': Value(dtype='int32', id=None)}


In [13]:
def modify_label(example):
    example['label'] = 0 if example['label'] == 0 else 1
    return example

train_data = train_data.map(modify_label)
val_m_data = val_m_data.map(modify_label)
val_mm_data = val_mm_data.map(modify_label)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

## Tokenizing the dataset


In [15]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [16]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.", padding="max_length", max_length=32)

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_val_m_data = val_m_data.map(tokenize_function, batched=True)
tokenized_val_mm_data = val_mm_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [18]:
example = tokenized_train_data[0]
print(example.keys())

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'])


In [19]:
tokenizer.decode(example['input_ids'])

'<s>Conceptually cream skimming has two basic dimensions - product and geography.</s></s>Product and geography are what make cream skimming work. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [20]:
example['label']

1

In [28]:
small_train_dataset = tokenized_train_data.shuffle(seed=42).select(range(1000))
small_val_dataset = tokenized_val_m_data.shuffle(seed=42).select(range(1000))

In [22]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model.to(DEVICE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [33]:
model_folder = base_dir + "Models-RoBERTa-" + str(datetime.datetime.now().timestamp())

if os.path.exists(model_folder) == False:
  os.mkdir(model_folder)


Trainer with arguments for saving and evaluating every epoch

In [24]:
metric_name = "accuracy"

args = TrainingArguments(
    output_dir = model_folder,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    resume_from_checkpoint=True,
    push_to_hub=True,
)

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Callback to add calcualtion of train accuracy (it is time-consuming though)

In [ ]:
class CustomCallback(TrainerCallback):

    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer

    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=small_train_dataset, #tokenized_train_data,
    eval_dataset=small_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#trainer.add_callback(CustomCallback(trainer))

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.638412,0.660000
2,No log,0.611314,0.660000
3,No log,0.617278,0.660000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

TrainOutput(global_step=96, training_loss=0.5981569687525431, metrics={'train_runtime': 89.2148, 'train_samples_per_second': 33.627, 'train_steps_per_second': 1.076, 'total_flos': 789333166080000.0, 'train_loss': 0.5981569687525431, 'epoch': 3.0})

In [ ]:
#trainer.train(resume_from_checkpoint=True)

Trainer with arguments for saving and evaluating every 5000 steps

In [34]:
new_args = TrainingArguments(
    output_dir = model_folder,
    evaluation_strategy = "steps",
    eval_steps=3068,
    save_strategy = "steps",
    save_steps=3068,
    logging_strategy="steps",
    logging_steps=3068,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    resume_from_checkpoint=True,
    push_to_hub=True,
)

new_trainer = Trainer(
    model,
    new_args,
    train_dataset=tokenized_train_data,
    eval_dataset=small_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [35]:
new_trainer.train()

Step,Training Loss,Validation Loss,Accuracy
3068,0.306500,0.245441,0.906000
6136,0.259300,0.256207,0.905000
9204,0.245200,0.224028,0.918000
12272,0.234400,0.210303,0.920000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

KeyboardInterrupt: 

In [36]:
new_trainer.state.log_history

[{'loss': 0.3065,
  'learning_rate': 1.7500000000000002e-05,
  'epoch': 0.25,
  'step': 3068},
 {'eval_loss': 0.24544084072113037,
  'eval_accuracy': 0.906,
  'eval_runtime': 7.411,
  'eval_samples_per_second': 134.934,
  'eval_steps_per_second': 4.318,
  'epoch': 0.25,
  'step': 3068},
 {'loss': 0.2593,
  'learning_rate': 1.5000000000000002e-05,
  'epoch': 0.5,
  'step': 6136},
 {'eval_loss': 0.25620728731155396,
  'eval_accuracy': 0.905,
  'eval_runtime': 7.4161,
  'eval_samples_per_second': 134.841,
  'eval_steps_per_second': 4.315,
  'epoch': 0.5,
  'step': 6136},
 {'loss': 0.2452, 'learning_rate': 1.25e-05, 'epoch': 0.75, 'step': 9204},
 {'eval_loss': 0.22402791678905487,
  'eval_accuracy': 0.918,
  'eval_runtime': 7.4177,
  'eval_samples_per_second': 134.813,
  'eval_steps_per_second': 4.314,
  'epoch': 0.75,
  'step': 9204},
 {'loss': 0.2344, 'learning_rate': 1e-05, 'epoch': 1.0, 'step': 12272},
 {'eval_loss': 0.2103027105331421,
  'eval_accuracy': 0.92,
  'eval_runtime': 7.4218

In [37]:
new_trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy
3068,0.306500,0.245441,0.906000
6136,0.259300,0.256207,0.905000
9204,0.245200,0.224028,0.918000
12272,0.234400,0.210303,0.920000
12597,0.234400,0.235702,0.920000


{'eval_loss': 0.23570236563682556, 'eval_accuracy': 0.92}

Saving the best model and sharing with Hugging Face community

In [38]:
new_trainer.push_to_hub('roberta-base-mnli-2-labels')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1704501036.30ca38a39465.1879.2:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/an-eve/Models-RoBERTa-1704501009.345538/commit/fdfd506644879431a1778f54c2072ac51b7b3bc1', commit_message='roberta-base-mnli-2-labels', commit_description='', oid='fdfd506644879431a1778f54c2072ac51b7b3bc1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#best_model_folder = base_dir + "RoBERTa-best"
#trainer.save_model(best_model_folder)

## Evaluate

Evaluating the model both on the validation matched and validation mismatched datasets

In [39]:
loaded_model = RobertaForSequenceClassification.from_pretrained("an-eve/roberta-base-mnli-2-labels")

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
loaded_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [40]:
args_test = TrainingArguments("Evaluation",
         per_device_eval_batch_size=BATCH_SIZE)

eval_trainer = Trainer(
    model=loaded_model,
    args=args_test,
    train_dataset=tokenized_val_m_data,
    eval_dataset=tokenized_val_m_data,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.2298658937215805,
 'eval_accuracy': 0.9207335710646969,
 'eval_runtime': 72.5948,
 'eval_samples_per_second': 135.203,
 'eval_steps_per_second': 4.229}

In [41]:
args_test = TrainingArguments("Evaluation",
         per_device_eval_batch_size=BATCH_SIZE)

eval_trainer = Trainer(
    model=loaded_model,
    args=args_test,
    train_dataset=tokenized_val_mm_data,
    eval_dataset=tokenized_val_mm_data,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.229410320520401,
 'eval_accuracy': 0.9208706265256306,
 'eval_runtime': 72.562,
 'eval_samples_per_second': 135.498,
 'eval_steps_per_second': 4.245}

Model's performance:

```
MNLI: 92.07%
MNLI-mm: 92.09%
```